In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

2021-12-09 11:56:05.102817: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-09 11:56:05.102839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pandas as pd
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import keras_tuner as kt

In [3]:
train_labels = []
train_samples = []

In [4]:
# read in the file
training_dataset = pd.read_csv(
    r'./waters_datasets/Cleaned_Datasets/Merged_Dataset_For_Trainging.csv',
    usecols=[
        "DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"
    ]
)[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"]]

x_train = training_dataset[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col"]]
y_train = training_dataset["WQI"]

In [5]:
x_train.head()

,DO,PH,Conductivity,BOD,NI,Fec_col,Tot_col
0,6.6,6.5,748.0,1.4,7.0,18.0,12000.0
1,6.6,6.4,338.0,1.1,0.7,75.0,5500.0
2,6.6,7.0,333.0,1.4,0.5,0.0,91.0
3,6.6,5.7,791.0,1.9,0.3,0.0,46.0
4,6.6,7.0,510.0,2.2,3.1,54.0,3350.0


In [6]:
y_train.head()

0    46.167450
1    40.826550
2    51.027703
3    32.954697
4    60.342890
Name: WQI, dtype: float64

In [7]:
# Testing dataset 1
testing_dataset_1 = pd.read_csv(r'./waters_datasets/Cleaned_Datasets/testing_data_1.csv',
    usecols=[
        "DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI",
    ]
)[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"]]

# testing cloumns for x 
x_test_1 = testing_dataset_1[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col"]]

# testing cloumns for y
y_test_1 = testing_dataset_1["WQI"]

In [8]:
#Use the hp argument to define the hyperparameters during model creation.
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
        activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256,  512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
   
    model.add(keras.layers.Dense(1, activation='linear'))
    model.compile(loss='mse')
    return model

In [9]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')


In [10]:
#add hyperparameter tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20)

tuner.search(x_train, y_train, epochs=500, validation_data=(x_test_1, y_test_1))
model = tuner.get_best_models()[0]

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


2021-12-09 11:51:37.520384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-09 11:51:37.520527: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-09 11:51:37.520638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-09 11:51:37.520746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-12-09 11:51:37.520854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [11]:
prediction = model.predict(x_test_1)

In [12]:
np.savetxt("foo.csv", prediction)

In [13]:
#Root mean square error
rmse = np.sqrt(mean_squared_error(y_test_1, prediction, squared=False))
print("RMSE: ", rmse)

RMSE:  12.109018484297001
